# Example Model Servers with Seldon

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [7]:
!helm install ../helm-charts/seldon-core-operator --name seldon-core  --set image.pullPolicy=IfNotPresent --set usageMetrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Sat Aug 31 10:58:33 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  1s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-config            1     1s
seldon-spartakus-config  1     1s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0         0/1    ContainerCreating  0         1s
seldon-spartakus-volunteer-7d6dd98f89-2jgbp  0/1    ContainerCreating  0         1s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     1s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10

In [10]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
Please note: There are reported gRPC issues with ambassador (see https://github.com/SeldonIO/seldon-core/issues/473).

In [6]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Sat Aug 31 10:56:45 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          1s

==> v1/Pod(related)
NAME                        READY  STATUS             RESTARTS  AGE
ambassador-5f7fd859b-4bhj7  0/1    ContainerCreating  0         0s
ambassador-5f7fd859b-6br5r  0/1    ContainerCreating  0         0s
ambassador-5f7fd859b-rr6pj  0/1    ContainerCreating  0         0s

==> v1/Service
NAME              TYPE          CLUSTER-IP      EXTERNAL-IP  PORT(S)                     AGE
ambassador        LoadBalancer  10.111.228.245  <pending>    80:31947/TCP,443:32642/TCP  1s
ambassador-admin  ClusterIP     10.99.206.49    <none>       8877/TCP                    1s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        1s

==> v1beta1/ClusterRole
NAME             AGE
ambassador       1s
ambassador-crds  1s

==> v1beta1/ClusterRoleBinding
NAME        

In [43]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


### Port Forward to Ambassador

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve SKlearn Iris Model

In [44]:
!pygmentize ../servers/sklearnserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG


In [50]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [51]:
!kubectl rollout status deploy/iris-default-4903e3c

Waiting for deployment "iris-default-4903e3c" rollout to finish: 0 of 1 updated replicas are available...
deployment "iris-default-4903e3c" successfully rolled out


In [52]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [53]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.44028212923599264
    values: 0.22694244373903638
    values: 0.08693601526817618
    values: 0.721446205469061
  }
}

Response:
meta {
  puid: "2k933j6cl1sq2kgh24ck96fmi8"
  requestPath {
    key: "classifier"
    value: "seldonio/sklearnserver_rest:0.2"
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  tensor {
    shape: 1
    shape: 3
    values: 0.3328333104192785
    values: 0.352243232066047
    values: 0.3149234575146744
  }
}



In [49]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In [8]:
!pygmentize ../servers/xgboostserver/samples/iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1


In [36]:
!kubectl apply -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [21]:
!kubectl rollout status deploy/iris-default-af1783b

Waiting for deployment "iris-default-af1783b" rollout to finish: 0 of 1 updated replicas are available...
deployment "iris-default-af1783b" successfully rolled out


In [22]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [23]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.2379841226697781
    values: 0.6766416345988621
    values: 0.8055747864263723
    values: 0.020395604144540558
  }
}

Response:
meta {
  puid: "gku8gaufrl69ra8btc1n1dih97"
  requestPath {
    key: "classifier"
    value: "seldonio/xgboostserver_rest:0.2"
  }
}
data {
  tensor {
    shape: 1
    values: 0.0
  }
}



In [37]:
!kubectl delete -f ../servers/xgboostserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model

**Will only work on a GCP Kubernetes Cluster**

In [14]:
!pygmentize ../servers/tfserving/samples/mnist_rest.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
    name: default
    replicas: 1


In [25]:
!kubectl apply -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [27]:
!kubectl rollout status deploy/mnist-default-4f27cf3

Waiting for deployment "mnist-default-4f27cf3" rollout to finish: 0 of 1 updated replicas are available...
deployment "mnist-default-4f27cf3" successfully rolled out


In [28]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

In [29]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.6960851737435524
    values: 0.31659351175832906
    values: 0.8839475487253241
    values: 0.3088659639767263
    values: 0.8891109022304765
    values: 0.19015731671987868
    values: 0.2084631221539136
    values: 0.4544208075311512
    values: 0.8926698161003384
    values: 0.19433553241548673
    values: 0.13531711500693755
    values: 0.8101485252410934
    values: 0.357998040732349
    values: 0.8149058111283166
    values: 0.7166138223407796
    values: 0.5481713626892256
    values: 0.8970834930204503
    values: 0.8913760644394269
    values: 0.8865946422300978
    values: 0.7283231572016693
    values: 0.061513546769038197
    values: 0.8946366478131781
    values: 0.08234186387502296
    values: 0.10724101325268287
    values: 0.6018581181011512
    values: 0.2833759431809676
    values: 0.12643611214180328
    values: 0.622302479013179
    values: 0.995550102668352
    values: 0.3328

In [30]:
!kubectl delete -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


## Serve MLFlow Elasticnet Wines Model

In [9]:
!pygmentize ../servers/mlflowserver/samples/elasticnet_wine.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1


In [3]:
!kubectl apply -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [6]:
!kubectl rollout status deploy/wines-default-057e362

deployment "wines-default-057e362" successfully rolled out


In [13]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [17]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.9370144045923416
    values: 0.8142853007643482
    values: 0.19471732807241948
    values: 0.5155919467447084
    values: 0.31446447854903803
    values: 0.6282233088869803
    values: 0.7504781874678473
    values: 0.6568746664582721
    values: 0.32870680648593553
    values: 0.09666979723379854
    values: 0.64486945913671
  }
}

Response:
meta {
  puid: "j2qdunn3uebeq5oj7gunmpr81u"
  requestPath {
    key: "classifier"
  }
}
data {
  tensor {
    shape: 1
    values: 5.231826975896663
  }
}



In [15]:
!kubectl delete -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
